In [ ]:
#loading libraries for importing and cleaning data
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

#training
from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from collections import defaultdict

In [ ]:
#import data
path = "/Users/hangxin/Documents/GitHub/Coupon_purchase/coupon-purchase-prediction/"
all_files = glob.glob(path + '/*.csv')
file_dict = {}
for file in all_files:
    file_dict[os.path.basename(file)[:-4]] = pd.read_csv(file)

In [ ]:
file_dict.keys()

In [ ]:
#to view the first three rows of each dataframe
for file,df in file_dict.items():
    print('The data frame is ',file)
    display(df.head(3))

### Exploratory Data Analysis

In [ ]:
file_dict['user_list'].info()

In [ ]:
#gender counts
sns.set_style('white')
plt.figure(figsize=(6,5))
sns.countplot(x='SEX_ID',data=file_dict['user_list'])
sns.despine()
plt.title('Gender Counts')

In general, there are more male users than female users. 

In [ ]:
#age distribution
plt.figure(figsize=(8,6))
sns.histplot(data=file_dict['user_list'], x='AGE',kde=True,bins=20,hue='SEX_ID')
sns.despine()
plt.title('User Age Distrition')

This plot suggests that most users are aged between 28 - 58 years old. 

For users younger than 25 years old, the numbers of female and male users are about the same. There are more female users aged between 25 to 45. In contrast, there are more male users older than 45.

Before exploring coupon lists for training and test, I'm going to translate the capsule and genre of each coupon from Japanese to English.

In [ ]:
def data_translation(df_input):
    df = df_input.copy(deep=True)
    #read in CAPSULE_TEXT_Translation file from documentation folder to translate Japanese to English
    capsule_text_translation1_path = '/Users/hangxin/Documents/GitHub/Coupon_purchase/coupon-purchase-prediction/documentation/capsule_text_translation1.xlsx'
    genre_name_translation_path = '/Users/hangxin/Documents/GitHub/Coupon_purchase/coupon-purchase-prediction/documentation/genre_name_translation.xlsx'
    
    capsule_text_translation1 = pd.read_excel(capsule_text_translation1_path,engine='openpyxl')
    genre_name_translation = pd.read_excel(genre_name_translation_path, engine='openpyxl')
    
    capsule_trans_df = dict(zip(capsule_text_translation1['CAPSULE_TEXT'],capsule_text_translation1['English Translation']))
    genre_trans_df = dict(zip(genre_name_translation['CAPSULE_TEXT'],genre_name_translation['English Translation']))
    
    #translate capsule text and genre to English
    df['capsule_text_eng'] = df['CAPSULE_TEXT'].apply(lambda x: capsule_trans_df[x])
    df['genre_name_eng'] = df['GENRE_NAME'].apply(lambda x: genre_trans_df[x])
    
    return df
    

In [ ]:
eng_coupon_list_train = data_translation(file_dict['coupon_list_train'])
eng_coupon_list_test = data_translation(file_dict['coupon_list_test'])

In [ ]:
#count the number of coupons usable on each day of week and holiday
usable_date_cnts = {}
for col in eng_coupon_list_train.iloc[:,11:20].columns:
    #values in these usable dates columns are 0,1,2
    #no further info provided for the meaning of 2, I assume it is usable
    usable_date_cnts[col] = len(np.where(eng_coupon_list_train[col]>0)[0])

#visualize the counts of usable days
keys = list(usable_date_cnts.keys())
vals = list(usable_date_cnts.values())

plt.figure(figsize=(9,6))
ax = sns.barplot(x=keys,y=vals, palette='crest')
ax.set_xticklabels(keys, rotation=45)
plt.title('Usable Dates of Coupons')

Coupons are usable mainly from Wednesday to Friday. Weekends are the least usable, followed by holidays and the day before. This makes sense as businesses that are having healthy sales over the weekends or holidays, aim to increase sales on weekdays. 

In [ ]:
#visualize the distribution of discount rates
plt.figure(figsize=(7,5))
ax = sns.histplot(eng_coupon_list_train['PRICE_RATE'],bins=60,color='lightblue')
ax.set_xlim([50,100]) #very few data points are on the left side of 50 
sns.despine()

plt.title('Distribution of Discount Rates')
plt.xlabel('Discount Rate')

Most discount rates are 50% of the original prices. 

In [ ]:
#visualize the distribution of original prices in Japanese yen
plt.figure(figsize=(7,5))
ax = sns.histplot(eng_coupon_list_train['CATALOG_PRICE'],
                  bins=200,color='lightblue')
ax.set_xlim([0,175000])
sns.despine()

plt.title('Distribution of Original Prices')
plt.xlabel('Original Price in Janpanese Yen')

In [ ]:
#joint plot to compare the origianl price and discount price
sns.jointplot(x='CATALOG_PRICE',y='DISCOUNT_PRICE',
              data=eng_coupon_list_train,height=8)


This joint plot compares the original catalog price to the discounted price. The largest graph indicates that outliers exist for the prices, while most prices are under

In [ ]:
#visualization of the prefecture names of shops
geometry = [Point(xy) for xy in zip(file_dict['prefecture_locations']['LONGITUDE'], file_dict['prefecture_locations']['LATITUDE'])]
gdf = GeoDataFrame(file_dict['prefecture_locations'], geometry=geometry) 
gdf = gdf.set_index('PREF_NAME').join(eng_coupon_list_train['ken_name'].value_counts())
gdf['ken_name_size'] = gdf['ken_name']/10

jp_map = gpd.read_file('/Users/hangxin/Documents/GitHub/Coupon_purchase/jpn_adm_2019_shp/jpn_admbndl_ALL_2019.shp')

fig, ax=plt.subplots(figsize=(15,15))

jp_map.plot(ax=ax,alpha=0.4, color='grey')
gdf.plot(ax=ax, color='blue', marker='o',markersize='ken_name_size')


The biggest blue spot indicates the largest number of coupons are from shops in Tokyo area. The second largest blue spot, southwest to Tokyo, suggests many other coupons are from shops in Osaka and Kyoto area. 

### Data Cleaning

**User list**

In [ ]:
file_dict['user_list'].info()

The above information of each column indicates some missing values in "PREF_NAME"(residential prefecture) and "WITHDRAW_DATE"(unregistered date). So I will add an "Unknown" to rows missing "PREF_NAME". If "WITHDRAW_DATE" is not null, this user unregistered from the website and will not be included in the prediction process.  

Also the "REG_DATE"(registered date) needs to be changed to date time format. Gender will be converted to categorical value.

In [ ]:
#dropping users who unregistered from the website
user_list_filtered = file_dict['user_list'].iloc[np.where(file_dict['user_list']['WITHDRAW_DATE'].isna())]

#drop WITHDRAW_DATE column as it is no longer useful
user_list_filtered = user_list_filtered.drop(columns='WITHDRAW_DATE')

#change null values to "Unknown" in PREF_NAME 
user_list_filtered['PREF_NAME'] = user_list_filtered['PREF_NAME'].fillna('Unknown')

#change REG_DATE to datetime formate
user_list_filtered['REG_DATE'] = user_list_filtered['REG_DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#create a column representing how long this user has been registered from the end of the training period
training_period = datetime.strptime('2012-06-23 11:59:00', '%Y-%m-%d %H:%M:%S')
user_list_filtered['num_year_registered'] = user_list_filtered['REG_DATE'].apply(lambda d:training_period.year-d.year)

#convert gender from string to categorical
user_list_filtered['SEX_ID'] = user_list_filtered['SEX_ID'].astype('category')


In [ ]:
user_list_filtered.info()

**Coupon lists**

Now I have a cleaned user list. I will also clean the coupon lists for training and test respectively.  

In [ ]:
eng_coupon_list_train.info()

From the summary of the coupon list for training, there are some null values for validity and usable dates and period. I want to find out why. 

In [ ]:
print('Null valid from dates by capsule:')
print(eng_coupon_list_train[eng_coupon_list_train['VALIDFROM'].isna()]['capsule_text_eng'].value_counts())
print('\n')

print('Null valid end dates by capsule:')
print(eng_coupon_list_train[eng_coupon_list_train['VALIDEND'].isna()]['capsule_text_eng'].value_counts())


In [ ]:
print('Null valid from dates by capsule in test set:')
print(eng_coupon_list_test[eng_coupon_list_test['VALIDFROM'].isna()]['capsule_text_eng'].value_counts())
print('\n')

print('Null valid end dates by capsule in test set:')
print(eng_coupon_list_test[eng_coupon_list_test['VALIDEND'].isna()]['capsule_text_eng'].value_counts())


In [ ]:
print('Null usable_date_mon by capsule in training set:')
print(eng_coupon_list_train[eng_coupon_list_train['USABLE_DATE_MON'].isna()]['capsule_text_eng'].value_counts())
print('\n')

print('Null usable_date_mon by capsule in test set:')
print(eng_coupon_list_test[eng_coupon_list_test['USABLE_DATE_MON'].isna()]['capsule_text_eng'].value_counts())


  The above results indicate the null values for validity dates and usable days are mainly for delivery services, lessons and gift cards, which are not time sensitive. Unlike food services, hotels or hair salons, the above services do not typically show heavier weekend or holiday traffic. So I will assume coupons from these services are valid from the release date to the end of training or test set period. And I will update the valid from and valid end dates correspondingly. Also I will assume these coupons are usable all days of week, including holidays and the day before, and update all usable dates to 1. 

In [ ]:
def parse_date_str(date_str, h_m_s):
    if h_m_s:
        new_date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    else:
        new_date = datetime.strptime(date_str, '%Y-%m-%d')
    return new_date

In [ ]:
def update_dates(df_input, type_of_dataset):
    df = df_input.copy(deep=True)
    
    #training and test period
    #training_start = datetime.strptime('2011-07-01' 12:00:00, '%Y-%m-%d %H:%M:%S')
    training_end = datetime.strptime('2012-06-23', '%Y-%m-%d')

    #test_start = datetime.strptime('2012-06-24', '%Y-%m-%d')
    test_end = datetime.strptime('2012-06-30', '%Y-%m-%d')

    #update valid start dates
    df['VALIDFROM'] = df.apply(lambda x: parse_date_str(x['DISPFROM'],True).date() if isinstance(x['VALIDFROM'], float) else x['VALIDFROM'], axis=1)
          
    #update end dates
    if type_of_dataset == 'train':
        #fill null values with training end date
        df['VALIDEND'] = df['VALIDEND'].fillna(training_end)
        
    if type_of_dataset == 'test':
        #fill null values with test end dates 
        df['VALIDEND'] = df['VALIDEND'].fillna(test_end)
    
    #update date type
    dispDates_cols = ['DISPFROM','DISPEND']
    for col in dispDates_cols:
        df[col] = df[col].apply(lambda d: parse_date_str(d,True))
    
    validDates_cols = ['VALIDFROM','VALIDEND']   
    for col in validDates_cols:
        df[col] = df[col].apply(lambda d: parse_date_str(d, False) if isinstance(d,str) else d)    
        
    #update valid period  
    df['VALIDPERIOD'] = df.apply(lambda x: (x['VALIDEND']-x['VALIDFROM']).days if isinstance(x['VALIDPERIOD'], float) else x['VALIDPERIOD'], axis=1)
    df['VALIDPERIOD'] = pd.to_numeric(df['VALIDPERIOD'])
    
    #update usable days to 1
    usable_days = ['USABLE_DATE_MON','USABLE_DATE_TUE','USABLE_DATE_WED','USABLE_DATE_THU',
                   'USABLE_DATE_FRI','USABLE_DATE_SAT','USABLE_DATE_SUN',
                  'USABLE_DATE_HOLIDAY','USABLE_DATE_BEFORE_HOLIDAY']
    for col in usable_days:
        df[col] = df[col].fillna(float(1))
            
    return df

In [ ]:
updated_coupon_list_train = update_dates(eng_coupon_list_train, 'train')
updated_coupon_list_test = update_dates(eng_coupon_list_test, 'test')

Now I have clean training and test sets of coupons. I want to examine if there is duplicate coupon in the test set.

In [ ]:
#examine if there is a coupon in both training and test sets
cnt = 0
for coupon in file_dict['coupon_list_test']['COUPON_ID_hash']:
    if coupon in file_dict['coupon_list_train']['COUPON_ID_hash']:
        cnt += 1

print(f'There are {cnt} coupons of test are also in training set',cnt)

In [ ]:
#selecting records of visits with no purchases as negative label(0: not purchase)
tmp_coupon_visit = file_dict['coupon_visit_train']
tmp_coupon_visit_no_purchase = tmp_coupon_visit[tmp_coupon_visit['PURCHASE_FLG']==0][["USER_ID_hash", "VIEW_COUPON_ID_hash", "I_DATE"]]
tmp_coupon_visit_no_purchase.rename(columns={'VIEW_COUPON_ID_hash':'COUPON_ID_hash'},inplace=True)
tmp_coupon_visit_no_purchase['label'] = 0

#selecting columns from coupon purchase table as positive label(1: purchase)
tmp_coupon_purchase = file_dict['coupon_detail_train'][["USER_ID_hash", "COUPON_ID_hash", "I_DATE"]]
tmp_coupon_purchase['label'] = 1


In [ ]:
# generate user coupon pair with label
user_coupon_train_pair = pd.concat([tmp_coupon_visit_no_purchase,tmp_coupon_purchase],
                                   ignore_index=True)

In [ ]:
#join the above dataframe with user features on user id
user_coupon_train_pair = user_coupon_train_pair.merge(user_list_filtered, on='USER_ID_hash')


In [ ]:
#then join coupon features on couple id
user_coupon_train_pair = user_coupon_train_pair.merge(updated_coupon_list_train, on='COUPON_ID_hash')

In [ ]:
user_coupon_train_pair.info()

Next, I will pre-processing the input features like one-hot encoding categorical values.

In [ ]:
#pre-process function
def pre_process_features(pair_df):
    #parsing the view date
    pair_df['I_DATE'] = pair_df['I_DATE'].apply(lambda x: parse_date_str(x, True))
    
    #create a new column which indicates whether the view date is within the discount price period
    pair_df['within_dispPeriod'] = pair_df.apply(lambda x:float(1) if x['DISPFROM'] <= x['I_DATE'] <= x['DISPEND'] else float(0), axis=1)

    #create a new column indicating whether the view date is within the coupon's valid period
    pair_df['within_validPeriod'] = pair_df.apply(lambda x:float(1) if x['VALIDFROM'] <= x['I_DATE'] <= x['VALIDEND'] else float(0), axis=1)
    
    #create a new column indicating whether this coupon is in the same area of user's residential prefecture
    pair_df['usr_cpn_same_area'] = pair_df.apply(lambda x: float(1) if (x['PREF_NAME']=='Unknown' or x['PREF_NAME']==x['ken_name']) else float(0), axis=1)

    #create users' age groups 
    pair_df['age_group'] = pd.cut(pair_df['AGE'],
                                bins=[0,22,34, 44, 64,120],
                                labels=['Youth', 'Young Adult','Early Mid-age','Late Mid-age','Senior Adult'],
                                right=False)
    return pair_df

In [ ]:
user_coupon_train_pair = pre_process_features(user_coupon_train_pair)

In [ ]:
#sort dataframe by date of view or purchase for cross validation split on a rolling basis
user_coupon_train_pair.sort_values(by='I_DATE', ignore_index=True)

In [ ]:
#one-hot encoding
X_categorical = user_coupon_train_pair[['SEX_ID', 'capsule_text_eng','age_group']]

one_hot_encoder = OneHotEncoder()
X_categorical_encoded = one_hot_encoder.fit_transform(X_categorical)

In [ ]:
#numeric features
dense_features = ['num_year_registered','PRICE_RATE',
                    'DISCOUNT_PRICE','DISPPERIOD','VALIDPERIOD','USABLE_DATE_MON',
                    'USABLE_DATE_TUE','USABLE_DATE_WED','USABLE_DATE_THU',
                   'USABLE_DATE_FRI','USABLE_DATE_SAT','USABLE_DATE_SUN',
                  'USABLE_DATE_HOLIDAY','USABLE_DATE_BEFORE_HOLIDAY','within_dispPeriod',
                    'within_validPeriod','usr_cpn_same_area']
X_dense = user_coupon_train_pair[dense_features]


In [ ]:
#merge numeric and categorical features together
X_train_all = np.hstack((X_dense.values,X_categorical_encoded.toarray()))

In [ ]:
#tscv = TimeSeriesSplit(n_splits=3)
#for train_index, validation_index in tscv.split(X_train):
    
 

In [ ]:
#take index from user_coupon_train_pair for training and validation 
train_index = np.where(user_coupon_train_pair['I_DATE']< datetime.strptime('2012-04-01 11:59:00', '%Y-%m-%d %H:%M:%S'))[0]
val_index = np.where(user_coupon_train_pair['I_DATE']>= datetime.strptime('2012-04-01 11:59:00', '%Y-%m-%d %H:%M:%S'))[0]

#training_perc = len(train_index[0])/len(user_coupon_train_pair)
#print('percentage of training data: ',training_perc)
#print('percentage of validation data: ',1-training_perc)


In [ ]:
train_index

In [ ]:
#apply index to X_train
y_train_all = user_coupon_train_pair['label']
X_train = X_train_all[train_index]
y_train = y_train_all[train_index]
X_val = X_train_all[val_index]
y_val = y_train_all[val_index]


In [ ]:
rfc = RandomForestClassifier(n_estimators=500, max_depth=7,class_weight='balanced', n_jobs=-1)
rfc.fit(X_train,y_train)

In [ ]:
y_train_all.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, rfc.predict(X_val))

In [ ]:
# read in data
#feval=rmsle,
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_val, y_val)
evals=[(dtrain, 'dtrain'), (dtest, 'dtest')]
# specify parameters via map
param = {'max_depth':5, 'eta':.1, 'objective':'binary:logistic', 'verbosity':2, "eval_metric": "auc" }
num_round = 200
bst = xgb.train(param, dtrain, num_round, evals=evals )
# make prediction
preds = bst.predict(dtest)

In [ ]:
#create a user_coupon_test_pair by cross joining clean user list and clean coupon test list

user_coupon_test_pair = pd.merge(user_list_filtered, updated_coupon_list_test,
                                on='key').drop('key',1)

In [ ]:
#assume all users have viewed all test coupons on the start date of test period
user_coupon_test_pair['I_DATE'] = '2012-06-24 12:00:00'

In [ ]:
n = len(user_coupon_test_pair)
test_pair_dict = {}

for i in range(50000,n,50000):
    last_i = i-50000
    if i <= 6800000:
        test_pair_dict[(last_i,i)] = pre_process_features(user_coupon_test_pair.iloc[last_i:i,:])

test_pair_dict[(i,n)] = pre_process_features(user_coupon_test_pair.iloc[i:,:])
        


In [ ]:
n = len(user_coupon_test_pair)
batch_size = 50000
user_coupon_dict = defaultdict(list)
#res = pd.DataFrame(columns=['USER_ID_hash','COUPON_ID_hash','preds'])
results = defaultdict(list)

for i in range(n // batch_size+1):
    start = i * batch_size
    end = (i+1) * batch_size
    batch_test_pair = pre_process_features(user_coupon_test_pair.iloc[start:end,:].copy())
    #extract the user_id and coupon_id
    user_coupon_id = user_coupon_test_pair[['USER_ID_hash','COUPON_ID_hash']][start:end]
    
    #one-hot encoding    
    batch_test_pair_encoded = one_hot_encoder.transform(batch_test_pair[categorical_features])

    #numeric features
    batch_test_pair_dense = batch_test_pair[dense_features]

    #merge numeric and categorical features together
    batch_X_test = np.hstack((batch_test_pair_dense.values,batch_test_pair_encoded.toarray()))

    #predicting
    batch_test_dmatrix = xgb.DMatrix(batch_X_test, label=None)
    predictions = bst.predict(batch_test_dmatrix)
    
    #combine user_id, coupon_id and corresponding predictions
    temp_res = user_coupon_id.assign(preds = predictions)
    
    for index,row in temp_res.iterrows():
        results[row['USER_ID_hash']].append((row['COUPON_ID_hash'],row['preds']))

In [ ]:
results = results.apply(lambda x: sort(x['']))

In [ ]:
for lst in results.values()

In [ ]:

categorical_features = ['SEX_ID', 'capsule_text_eng','age_group']
pred_dict = {}
for key, batch_test_pair in test_pair_dict.items():
    #one-hot encoding    
    batch_test_pair_encoded = one_hot_encoder.transform(batch_test_pair[categorical_features])

    #numeric features
    batch_test_pair_dense = batch_test_pair[dense_features]

    #merge numeric and categorical features together
    X_test_batch = np.hstack((batch_test_pair_dense.values,batch_test_pair_encoded.toarray()))

#predicting
    test_dmatrix = xgb.DMatrix(X_test_batch, label=None)
    pred_dict[key] = bst.predict(test_dmatrix)

In [ ]:
list(pred_dict.values())[0]

### Model